In [ ]:
!pip install pyspark

In [7]:
import pyspark
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.appName('MLlib').getOrCreate()
spark

In [11]:
training = spark.read.csv('test1.csv', header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [13]:
training.printSchema(), training.columns

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



(None, ['Name', 'age', 'Experience', 'Salary'])

## create new column using two or more column combination

In [14]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol='Independent features')

In [15]:
output = featureassembler.transform(training)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [16]:
final_data = output.select('Independent features', 'Salary')
final_data.show()

+--------------------+------+
|Independent features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression

train_data, test_data = final_data.randomSplit([0.75,0.25])

regressor = LinearRegression(featuresCol='Independent features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [19]:
regressor.coefficients

DenseVector([-0.0, 1666.6667])

In [20]:
pred = regressor.evaluate(test_data)
pred.predictions.show()

+--------------------+------+------------------+
|Independent features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|16666.666666666697|
|          [24.0,3.0]| 20000|18333.333333333365|
|          [30.0,8.0]| 25000|26666.666666666664|
+--------------------+------+------------------+



In [21]:
pred.meanAbsoluteError, pred.meanSquaredError, pred.r2

(1555.555555555534, 2444444.44444438, 0.7179487179487254)